<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Support Vector Machine**


Estimated time needed: **30** minutes


In this lab, you will learn and obtain hands-on practices on Support Vector Machine model.


We will be using a real-world diabetes food items suggestion dataset, which contains detailed nutrition information about a food item. The objective is to classify what food a diabetic patient should choose More Often or Less Often for a specific food item given its nutrients.


## Objectives


After completing this lab you will be able to:


*   Train and evaluate SVM classifiers
*   Tune important SVM hyperparameters such as regularization and kernel types
*   Plot hyperplanes and margins from trained SVM models


## SVM Overview


SVM tries to find hyperplanes that have the maximum margin. The hyperplanes are determined by support vectors (data points have the smallest distance to the hyperplanes). Meanwhile, in order to reduce model variance, the SVM model aims to find the maximum possible margins so that unseen data will be more likely to be classified correctly.


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/module3_svm/images/svm.png" width="720" alt="evaluation metrics"  />
</center>


SVM addresses non-linear separable via kernel trick. Kernels are a special type of function that takes two vectors and returns a real number, like a dot-product operation. As such, kernels are not any real mapping functions from low dimensional spaces to high dimensional spaces.


For example, suppose we have two vectors $x = (x\_{1}, x\_{2})$ and $y = (y\_{1}, y\_{2})$


Now we have a simple polynomial kernel like the following:


$$k(x, y) = (x^Ty)^2$$


If we apply the kernel on vector `x` and `y`, we will get:


$$k(x, y) = (x^Ty)^2 = (x\_{1}y\_{1} + x\_{2}y\_{2})^2 = x\_{1}^2y\_{1}^2 + x\_{2}^2y\_{2}^2 + 2x\_{1}x\_{2}y\_{1}y\_{2}$$


It can be seen as a dot-product between two higher-dimensional vectors (`3-dimensional`):


$$\hat{x} = (x\_{1}^2, x\_{2}^2, \sqrt{2}x\_{1}x\_{2}) $$


$$\hat{y} = (y\_{1}^2, x\_{2}^2, \sqrt{2}y\_{1}y\_{2})$$


As such, computing the `k(x, y)` is equivalent to computing a dot-product of the higher dimensional vectors, without doing the actual feature space transforms. Consequently, SVM with non-linear kernels can transform existing features into high dimensional features that can be linearly separated in higher dimensional spaces.


***


## Setup lab environment


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy pandas==1.3.3 numpy==1.21.2 ipywidgets==7.4.2 scipy==7.4.2 tqdm==4.62.3 matplotlib==3.5.0 seaborn==0.9.0

# install imbalanced-learn package
!pip install imbalanced-learn==0.8.0
# Note: If your environment doesn't support "!mamba install", use "!pip install" 

In [ ]:
# Import required packages
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
# Evaluation metrics related methods
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_recall_fscore_support, precision_score, recall_score

import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
%matplotlib inline

In [ ]:
# Setup a random seed to be 123
rs = 123

## Load and explore dataset


Let's first load the dataset as a `Pandas` dataframe and conduct some basic explorations


In [ ]:
# Load the dataset
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/datasets/food_items_binary.csv"
food_df = pd.read_csv(dataset_url)

and let's quickly looks at its first 5 rows


In [ ]:
food_df.head(10)

In [ ]:
# Get the row entries with col 0 to -1 (16)
feature_cols = list(food_df.iloc[:, :-1].columns)
feature_cols

In [ ]:
X = food_df.iloc[:, :-1]
y = food_df.iloc[:, -1:]

In [ ]:
X.describe()

as we can see from the outputs above, this food item dataset contains 17 types of nutrients about a food item such as Calories, Total Fat, Protein, Sugar, and so on, as numeric variables.


Next, let's check the target variable, such as the `class` column to see what are label values and their distribution.


In [ ]:
# # Get the row entries with the last col 'class'
y.value_counts(normalize=True)

In [ ]:
y.value_counts().plot.bar(color=['red', 'green'])

As we can see from the bar chart above, this dataset has two classes `Less Often` and `More Often`. The two labels are imbalanced with most food items should be chosen less often for diabetic patients.


## Build a SVM model with default parameters


First, let's split the training and testing dataset. Training dataset will be used to train and tune models, and testing dataset will be used to evaluate the models. Note that you may also split a validation dataset from the training dataset for model tuning only.


In [ ]:
# First, let's split the training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = rs)

Okay, now we have the training and testing datasets ready, let's start the model training task.


We first define a `sklearn.svm import SVC` model with all default arguments.


In [ ]:
model = SVC()

Then train the model with training dataset:


In [ ]:
model.fit(X_train, y_train.values.ravel())

and make predictions


In [ ]:
preds = model.predict(X_test)

Here we defined a utility method to evaluate the model performance.


In [ ]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp, average='binary')
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

In [ ]:
evaluate_metrics(y_test, preds)

As we can see from the evaluation results above, the default SVM model achieves relatively good performance on this binary classification task. The overall accuracy is around `0.95` and the f1score is around `0.82`.


Now, you have easily built a SVM model with relatively good performance. Can we achieve better classification performance by customizing the model?


## Train SVM with different regularization parameters and kernels


The `SVC` model provided by sklearn has two important arguments to be tuned: regularization parameter `C` and `kernel`.


The `C` argument is a regularization parameter.

*   For large values of C, the optimization will choose a smaller-margin hyperplane if that hyperplane does a better job of getting all the training points classified correctly.
*   Conversely, a very small value of C will cause the optimizer to look for a larger-margin separating hyperplane, even if that hyperplane misclassifies more points.


The `kernel` argument specifies the kernel to be used for transforming features to higher-dimensional spaces, some commonly used non-linear kernels are:

*   `rbf`: Gaussian Radial Basis Function (RBF)
*   `poly`: Polynomial Kernel
*   `sigmoid`: Sigmoid Kernel


Let's first try `C = 10` and ` kernel = 'rbf'  `


In [ ]:
model = SVC(C=10, kernel='rbf')
model.fit(X_train, y_train.values.ravel())
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)

You should see that we have better performance than the default SVM model trained in the previous step. Now, it's your turn to try different parameters yourself.


### Coding Exercise: Try different `C` values and `kernels` to see which combination produces SVM models with better classification performance.


In [ ]:
# Type your code here

## Tune regularization parameter C and Kernels via GridSearch


Exhaustively trying different hyperparameters by hands is infeasible. Thus, `sklearn` provides users with many automatic hyperparameter tuning methods. A popular one is grid search cross-validation `GridSearchCV`


Next, let's quickly try `GridSearchCV` to find the optimized `C` and `kernel` combination:


We first define some candidate parameter values we want to search in a `dict` object, like the following setting:


In [ ]:
params_grid = {
    'C': [1, 10, 100],
    'kernel': ['poly', 'rbf', 'sigmoid']
}

Then, we define a  SVM model


In [ ]:
model = SVC()

and use create a `GridSearchCV` method to grid search `params_grid` and find the optimized combination with best `f1` score. The searching process may take several minutes to complete.


In [ ]:
# Define a GridSearchCV to search the best parameters
grid_search = GridSearchCV(estimator = model, 
                           param_grid = params_grid, 
                           scoring='f1',
                           cv = 5, verbose = 1)
# Search the best parameters with training data
grid_search.fit(X_train, y_train.values.ravel())
best_params = grid_search.best_params_

In [ ]:
best_params

Okay, we can see `C=100` and `kernel=`rbf\`\` seems to produce the highest f1score. Let's quickly try this combination to see the model performance.


In [ ]:
model = SVC(C=100, kernel='rbf')
model.fit(X_train, y_train.values.ravel())
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)

The best f1score now becomes `0.88` after hyperparameter tuning.


## Plot SVM hyperplane and margin


Okay, you have learned how to define, train, evaluate, and fine-tune a SVM model with `sklearn`. However, so far we only obtained plain evaluation metrics and they are not intuitive to help us understand and interpret an SVM model. It would be great to visualize the see actual hyperplains and margins learned in an SVM model.


Since it is challenging to visualize a hyperplane higher than 3 dimensions. To illustrate the idea, we will focus on a 2-dimensional feature space.


We first simplify the dataset with only two features `Calories` and `Dietary Fiber`, and include only 1000 instances:


In [ ]:
simplified_food_df = food_df[['Calories', 'Dietary Fiber', 'class']]

In [ ]:
X = simplified_food_df.iloc[:1000, :-1].values
y = simplified_food_df.iloc[:1000, -1:].values

and we undersample the majority class `Class = 0` to balance the class distribution so we will produce a clearer visualization.


In [ ]:
under_sampler = RandomUnderSampler(random_state=123)
X_under, y_under = under_sampler.fit_resample(X, y)

In [ ]:
print(f"Dataset resampled shape, X: {X_under.shape}, y: {y_under.shape}")

To better show the hyperplane and margins, we normalize the features with a `MinMaxScaler`.


In [ ]:
scaler = MinMaxScaler()
X_under = scaler.fit_transform(X_under)

Okay, let's first train a linear SVM model with `kernel=linear` so that we can get a linear hyperplane and margins.


In [ ]:
linear_svm = SVC(C=1000, kernel='linear')
linear_svm.fit(X_under, y_under)

Here we also provided an utility method to plot the decision boundary (hyperplane), support vectors, and margins. You may write your own visualization method if you are interested.


In [ ]:
def plot_decision_boundry(X, y, model):
    plt.figure(figsize=(16, 12))
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired)

    # plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = model.decision_function(xy).reshape(XX.shape)

    # plot decision boundary and margins
    ax.contour(
        XX, YY, Z, colors="k", levels=[-1, 0, 1], alpha=0.5, linestyles=["--", "-", "--"]
    )

    # plot support vectors
    ax.scatter(
        model.support_vectors_[:, 0],
        model.support_vectors_[:, 1],
        s=100,
        linewidth=1,
        facecolors="none",
        edgecolors="k",
    )
    plt.show()

In [ ]:
plot_decision_boundry(X_under, y_under, linear_svm)

Okay, we can see a clear linear hyperplane seperates the two classes (Blue dots vs Orange dots). The highlighted dots are the support vectors determining the hyperplain.


If we want to include non-linear kernels, we should get a non-linear decision boundary in the 2-d space (maybe linear in higher feature space). So here we use a `rbf` kernel:


In [ ]:
svm_rbf_kernel = SVC(C=100, kernel='rbf')
svm_rbf_kernel.fit(X_under, y_under)

In [ ]:
plot_decision_boundry(X_under, y_under, svm_rbf_kernel)

We now see a non-linear hyperplane and margins separating the two classes.


### Coding Exercise: Try different `C` values and `kernels` to see the how they affect the hyperplanes and margins.


In [ ]:
## Type your code here


<details><summary>Click here for a sample solution</summary>

```python
svm_rbf_kernel = SVC(C=100, kernel='poly')
svm_rbf_kernel.fit(X_under, y_under)
plot_decision_boundry(X_under, y_under, svm_rbf_kernel)
```

</details>


## Next Steps


Great! Now you have learned and practiced SVM model and applied it to solve a real-world food classification problem for diabetic patients. You also learned how to visualize the hyperplanes and margins generated by the SVM models.


Next, you will be learning other popular classification models with different structures, assumptions, cost functions, and application scenarios.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2021-01-01)


### Other Contributors


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-9-23         | 1.0     | Yan        | Created the initial version |
| 2022-2-9          | 1.1     | Steve Hord | QA pass                     |


Copyright © 2021 IBM Corporation. All rights reserved.
